## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob

In [30]:
pwd

'd:\\AIHUB'

### Function

In [3]:
def json_file_name_list(path_list):
    for i in path_list:
        file_name = glob(i, recursive = True)
    return file_name

In [4]:
def train_valid_json_file_name_list(path_list):
    for i in path_list:
        if 'rain' in i:
            train_file_name = glob(i, recursive = True)
        elif 'alid' in i:  
            valid_file_name = glob(i, recursive = True)
    return train_file_name, valid_file_name

In [11]:
def divide_source_file_list(l, n): 
    for i in range(0, len(l), n): 
        yield l[i:i + n] 

In [5]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  text_file_name_list = []

  for i in range(len(source_file_nested_list)):
    txt_file_name = folder_corpus_type_name + str(i) + ".txt"
    text_file_name_list.append(txt_file_name)

  return text_file_name_list

In [6]:
def train_valid_txt_file_name_list(source_file_nested_list, folder_corpus_type_name):
   
  train_valid_set = source_file_nested_list[0][0]
  text_file_name_list = []

  if 'rain' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)

  elif 'alid' in train_valid_set:
    for i in range(len(source_file_nested_list)):
      txt_file_name = folder_corpus_type_name + str(i) + ".txt"
      text_file_name_list.append(txt_file_name)  

  return text_file_name_list

In [7]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

In [8]:
def informal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거
    for sentence in kss.split_sentences(source, use_heuristic=True,
                                        num_workers=32):
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # The String starts with a letter
            # The String ends with [. ! ?]
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 요약문 및 레포트 생성 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582)

In [32]:
path_list = ['AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/07.public/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/10.narration/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/07.public/' + '**/*.json',             
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/10.narration/' + '**/*.json']

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
valid_file_name_01 = glob(path_list[10], recursive = True)
valid_file_name_02 = glob(path_list[11], recursive = True)
valid_file_name_03 = glob(path_list[12], recursive = True)
valid_file_name_04 = glob(path_list[13], recursive = True)
valid_file_name_05 = glob(path_list[14], recursive = True)
valid_file_name_06 = glob(path_list[15], recursive = True)
valid_file_name_07 = glob(path_list[16], recursive = True)
valid_file_name_08 = glob(path_list[17], recursive = True)
valid_file_name_09 = glob(path_list[18], recursive = True)
valid_file_name_10 = glob(path_list[19], recursive = True)  

In [34]:
the_number_of_file = len(train_file_name_01) 
print("The number of file: ", the_number_of_file)
n = the_number_of_file // 1000
print("The number of list element:", n)
train_file_nested_list_01 = list(divide_source_file_list(train_file_name_01, n))
train_file_nested_list_02 = list(divide_source_file_list(train_file_name_02, n))
train_file_nested_list_03 = list(divide_source_file_list(train_file_name_03, n))
train_file_nested_list_04 = list(divide_source_file_list(train_file_name_04, n))
train_file_nested_list_05 = list(divide_source_file_list(train_file_name_05, n))
train_file_nested_list_06 = list(divide_source_file_list(train_file_name_06, n))
train_file_nested_list_07 = list(divide_source_file_list(train_file_name_07, n))
train_file_nested_list_08 = list(divide_source_file_list(train_file_name_08, n))
train_file_nested_list_09 = list(divide_source_file_list(train_file_name_09, n))
train_file_nested_list_10 = list(divide_source_file_list(train_file_name_10, n))
valid_file_nested_list_01 = list(divide_source_file_list(valid_file_name_01, n))
valid_file_nested_list_02 = list(divide_source_file_list(valid_file_name_02, n))
valid_file_nested_list_03 = list(divide_source_file_list(valid_file_name_03, n))
valid_file_nested_list_04 = list(divide_source_file_list(valid_file_name_04, n))
valid_file_nested_list_05 = list(divide_source_file_list(valid_file_name_05, n))
valid_file_nested_list_06 = list(divide_source_file_list(valid_file_name_06, n))
valid_file_nested_list_07 = list(divide_source_file_list(valid_file_name_07, n))
valid_file_nested_list_08 = list(divide_source_file_list(valid_file_name_08, n))
valid_file_nested_list_09 = list(divide_source_file_list(valid_file_name_09, n))
valid_file_nested_list_10 = list(divide_source_file_list(valid_file_name_10, n))

The number of file:  21600
The number of list element: 21


In [35]:
train_text_file_name_list_01 = train_valid_txt_file_name_list(train_file_nested_list_01,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_01.news_r_")
train_text_file_name_list_02 = train_valid_txt_file_name_list(train_file_nested_list_02,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_02.briefing_")
train_text_file_name_list_03 = train_valid_txt_file_name_list(train_file_nested_list_03,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_03.his_cul_")
train_text_file_name_list_04 = train_valid_txt_file_name_list(train_file_nested_list_04,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_04.paper_")
train_text_file_name_list_05 = train_valid_txt_file_name_list(train_file_nested_list_05,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_05.minute_")
train_text_file_name_list_06 = train_valid_txt_file_name_list(train_file_nested_list_06,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_06.edit_")
train_text_file_name_list_07 = train_valid_txt_file_name_list(train_file_nested_list_07,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_07.public_")
train_text_file_name_list_08 = train_valid_txt_file_name_list(train_file_nested_list_08,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_08.speech_")
train_text_file_name_list_09 = train_valid_txt_file_name_list(train_file_nested_list_09,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_09.literature_")
train_text_file_name_list_10 = train_valid_txt_file_name_list(train_file_nested_list_10,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_10.narration_")
valid_text_file_name_list_01 = train_valid_txt_file_name_list(valid_file_nested_list_01,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_01.news_r_")
valid_text_file_name_list_02 = train_valid_txt_file_name_list(valid_file_nested_list_02,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_02.briefing_")
valid_text_file_name_list_03 = train_valid_txt_file_name_list(valid_file_nested_list_03,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_03.his_cul_")
valid_text_file_name_list_04 = train_valid_txt_file_name_list(valid_file_nested_list_04,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_04.paper_")
valid_text_file_name_list_05 = train_valid_txt_file_name_list(valid_file_nested_list_05,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_05.minute_")
valid_text_file_name_list_06 = train_valid_txt_file_name_list(valid_file_nested_list_06,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_06.edit_")
valid_text_file_name_list_07 = train_valid_txt_file_name_list(valid_file_nested_list_07,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_07.public_")
valid_text_file_name_list_08 = train_valid_txt_file_name_list(valid_file_nested_list_08,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_08.speech_")
valid_text_file_name_list_09 = train_valid_txt_file_name_list(valid_file_nested_list_09,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_09.literature_")
valid_text_file_name_list_10 = train_valid_txt_file_name_list(valid_file_nested_list_10,
                                                               "demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_10.narration_")

the_numer_of_txt_file = len(train_text_file_name_list_01) + len(train_text_file_name_list_02) + \
    len(train_text_file_name_list_03) + len(train_text_file_name_list_04) + \
    len(train_text_file_name_list_05) + len(train_text_file_name_list_06) + \
    len(train_text_file_name_list_07) + len(train_text_file_name_list_08) + \
    len(train_text_file_name_list_09) + len(train_text_file_name_list_10) +  \
    len(valid_text_file_name_list_01) + len(valid_text_file_name_list_02) + \
    len(valid_text_file_name_list_03) + len(valid_text_file_name_list_04) + \
    len(valid_text_file_name_list_05) + len(valid_text_file_name_list_06) + \
    len(valid_text_file_name_list_07) + len(valid_text_file_name_list_08) + \
    len(valid_text_file_name_list_09) + len(valid_text_file_name_list_10)
print("The number of txt file:", the_numer_of_txt_file )

The number of txt file: 7868


In [36]:
def make_corpus_txt(source_file_nested_list, text_file_name_list):

  print("[Size]")
  print("The numnber of preprocessing corpus: " + str(len(source_file_nested_list)))
  print("\n[Order]")
  num = 0
  for source_file_list, txt_file_name in zip(source_file_nested_list, text_file_name_list):
    sentence_list = []

    num += 1
    print(str(num), end=" ")
    
    for i in range(len(source_file_list)):

      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) != str:
        source_file = source_file_list[i]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source = one_json_sample['Meta(Refine)']['passage']
      sentences = formal_preprocessing_text(source)
      for sentence in sentences:
        sentence_list.append(sentence)   
          
    with open(os.path.join('AIHUB_corpus/', txt_file_name), "a", encoding='utf-8') as fp:        
        fp.write("\n".join(sentence_list))        

In [ ]:
make_corpus_txt(train_file_nested_list_01, train_text_file_name_list_01)

In [ ]:
make_corpus_txt(train_file_nested_list_02, train_text_file_name_list_02)

In [ ]:
make_corpus_txt(train_file_nested_list_03, train_text_file_name_list_03)

In [ ]:
make_corpus_txt(train_file_nested_list_04, train_text_file_name_list_04)

In [ ]:
make_corpus_txt(train_file_nested_list_05, train_text_file_name_list_05)

In [ ]:
make_corpus_txt(train_file_nested_list_06, train_text_file_name_list_06)

In [ ]:
make_corpus_txt(train_file_nested_list_07, train_text_file_name_list_07)

In [ ]:
make_corpus_txt(train_file_nested_list_08, train_text_file_name_list_08)

In [ ]:
make_corpus_txt(train_file_nested_list_09, train_text_file_name_list_09)

In [ ]:
make_corpus_txt(train_file_nested_list_10, train_text_file_name_list_10)

In [ ]:
make_corpus_txt(valid_file_nested_list_01, valid_text_file_name_list_01)

In [ ]:
make_corpus_txt(valid_file_nested_list_02, valid_text_file_name_list_02)

In [ ]:
make_corpus_txt(valid_file_nested_list_03, valid_text_file_name_list_03)

In [ ]:
make_corpus_txt(valid_file_nested_list_04, valid_text_file_name_list_04)

In [ ]:
make_corpus_txt(valid_file_nested_list_05, valid_text_file_name_list_05)

In [ ]:
make_corpus_txt(valid_file_nested_list_06, valid_text_file_name_list_06)

In [ ]:
make_corpus_txt(valid_file_nested_list_07, valid_text_file_name_list_07)

In [ ]:
make_corpus_txt(valid_file_nested_list_08, valid_text_file_name_list_08)

In [ ]:
make_corpus_txt(valid_file_nested_list_09, valid_text_file_name_list_09)

In [ ]:
make_corpus_txt(valid_file_nested_list_10, valid_text_file_name_list_10)

In [ ]:
corpus_list = glob("AIHUB_corpus/demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_train_" +"*.txt")
corpus_list += glob("AIHUB_corpus/demo/summary_and_report_generation_data/AIHUB_summary_and_report_generation_data_valid_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/demo/AIHUB_summary_and_report_generation_data.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)